# Grating coupler

Based on tidy3D example notebook on [github](https://github.com/flexcompute/tidy3d-notebooks/blob/main/GratingCoupler.ipynb)


In [ ]:
# get the most recent version of tidy3d
#pip install -q --upgrade tidy3d

# make sure notebook plots inline
%matplotlib inline

# basic imports
import numpy as np
import matplotlib.pylab as plt

# tidy3d imports
import tidy3d as td
import tidy3d.web as web

## Problem Setup

In this example, we model a 3D grating coupler in a Silicon on Insulator (SOI) platform.

A basic schematic of the design is shown below.  The simulation is about 19um x 4um x 5um with a wavelength of 1.55um and takes about 1 minute to simulate 10,000 time steps.

In the simulation, we inject a modal source into the waveguide and propagate it towards the grating structure.  The radiation from the grating coupler is then measured with a near field monitor and we use a far field projection to inspect the angular dependence of the radiation.


<img src="img/grating_coupler.png" width=600>

In [ ]:
# basic parameters (note, all length units are microns)
nm = 1e-3
wavelength = 1550 * nm

# resolution
grids_per_wavelength = 50.0
dl = wavelength / grids_per_wavelength

# waveguide
wg_width = 400 * nm
wg_height = 220 * nm
wg_length = 2 * wavelength

# surrounding
sub_height = 2.0
clad_height = 2.0
buffer = 0.5 * wavelength

# coupler
cp_width = 2 * wavelength
cp_length = 10 * wavelength
taper_length = 4 * wavelength

In [ ]:
# sizes
Lx = buffer + wg_length + taper_length + cp_length
Ly = buffer + cp_width + buffer
Lz = sub_height + wg_height + clad_height
sim_size = [Lx, Ly, Lz]

# convenience variables to store center of coupler and waveguide
wg_center_x = +Lx/2 - buffer - (wg_length + taper_length)/2
cp_center_x = -Lx/2 + buffer + cp_length/2
wg_center_z = -Lz/2 + sub_height + wg_height/2
cp_center_z = -Lz/2 + sub_height + wg_height/2

# materials
Clad = td.Medium(epsilon=1.44**2)
Si = td.Medium(epsilon=3.47**2)
SiO2 = td.Medium(epsilon=1.44**2)

# source parameters
freq0 = td.constants.C_0 / wavelength
fwidth = freq0 / 10
run_time = 100 / fwidth

# PML layers
npml = 15

## Mode Solve

To determine the pitch of the waveguide for a given design angle, we need to compute the effective index of the waveguide mode being coupled into.  For this, we set up a simple simulation of the coupler region and use the mode solver to get the effective index. We will not run this simulation, we just add a ``ModeMonitor`` object in order to call the mode solver, ``sim.compute_modes()`` below, and get the effective index of the wide-waveguide region.

In [ ]:
# grating parameters
design_theta_deg = 30
design_theta_rad = np.pi * design_theta_deg / 180
grating_height = 70 * nm

# do a mode solve to get neff of the coupler

sub = td.Box(
    center=[0, 0, -Lz/2],
    size=[td.inf, td.inf, 2 * sub_height],
    material=SiO2,
    name='substrate')

clad = td.Box(
    center=[0, 0, Lz/2],
    size=[td.inf, td.inf, td.inf],
    material=SiO2,
    name='clad')

cp = td.Box(
    center=[0, 0, cp_center_z-grating_height/4],
    size=[td.inf, cp_width, wg_height-grating_height/2],
    material=Si,
    name='coupler')

mode_mnt = td.ModeMonitor(
    center=[0, 0, 0],
    size=[0, 8*cp_width, 8*wg_height],
    freqs=freq0)

sim = td.Simulation(
    size=sim_size,
    mesh_step=[dl, dl, dl],
    structures=[clad, sub, cp],
    sources=[],
    monitors=[mode_mnt],
    run_time=run_time,
    pml_layers=[npml, npml, npml])

sim.viz_mat_2D(normal='x');

In [ ]:
sim.viz_eps_2D(normal='x', cbar=True)

In [ ]:
# Compute and visualize the first two modes of the monitor
sim.compute_modes(mode_mnt, Nmodes=2)
sim.viz_modes(mode_mnt)

# Get the data for the mode corresponding to frequency index 0 and mode index 0
mode = sim.data(mode_mnt)["modes"][0][0]
neff = mode.neff
print(f'\n\neffective index of coupler region, mode 0 = {neff}')

## Create Simulation
Now we set up the grating coupler to simulate in Tidy3D.

In [ ]:
# gratings
design_theta_deg = 10
design_theta_rad = np.pi * design_theta_deg / 180
pitch = wavelength / (neff - np.sin(design_theta_rad))
grating_length = pitch / 2.0
num_gratings = int(cp_length / pitch)

sub = td.Box(
    center=[0, 0, -Lz/2],
    size=[td.inf, td.inf, 2 * sub_height],
    material=SiO2,
    name='substrate')

wg = td.Box(
    center=[wg_center_x, 0, wg_center_z],
    size=[buffer + wg_length + taper_length + cp_length/2, wg_width, wg_height],
    material=Si,
    name='waveguide')

cp = td.Box(
    center=[cp_center_x, 0, cp_center_z],
    size=[cp_length, cp_width, wg_height],
    material=Si,
    name='coupler')

tp = td.PolySlab(
    vertices=[
        [cp_center_x + cp_length/2 + taper_length, + wg_width/2],
        [cp_center_x + cp_length/2 + taper_length, - wg_width/2],
        [cp_center_x + cp_length/2, - cp_width/2],
        [cp_center_x + cp_length/2, + cp_width/2]],
    z_cent=wg_center_z,
    z_size=wg_height,
    material=Si,
    name='taper')

grating_left_x = cp_center_x - cp_length/2
gratings = [
    td.Box(
        center=[grating_left_x + (i + 0.5) * pitch, 0, cp_center_z + wg_height/2 - grating_height/2],
        size=[grating_length, cp_width, grating_height],
        material=Clad,
        name=f'{i}th_grating')
    for i in range(num_gratings)]

In [ ]:
mode_source = td.ModeSource(
    td.GaussianPulse(freq0, fwidth, phase=0),
    center=[Lx/2 - buffer, 0, cp_center_z],
    size=[0, 8*wg_width, 8*wg_height],
    direction='backward',
    amplitude=1.0,
    name='modal_source')

In [ ]:
# distance to near field monitor
nf_offset = 50 * nm

plane_monitor = td.FreqMonitor(
    center=[0, 0, cp_center_z],
    size=[Lx, Ly, 0],
    freqs=freq0,
    name='full_domain_fields')

rad_monitor = td.FreqMonitor(
    center=[0, 0, 0],
    size=[Lx, 0, Lz],
    freqs=freq0,
    name='full_domain_fields')

near_field_monitor = td.FreqMonitor(
    center=[cp_center_x, 0, cp_center_z + wg_height/2 + nf_offset],
    size=[cp_length, cp_width, 0],
    freqs=freq0,
    name='radiated_near_fields')

In [ ]:
sim = td.Simulation(
    size=sim_size,
    mesh_step=[dl, dl, dl],
    structures=[sub, wg, cp, tp] + gratings,
    sources=[mode_source],
    monitors=[plane_monitor, rad_monitor, near_field_monitor],
    run_time=run_time,
    pml_layers=[npml, npml, npml])

In [ ]:
fig, axes = plt.subplots(1, 3, tight_layout=True, figsize=(14, 3))
for val, pos, ax in zip('xyz', (0, 0, -Lz/2 + sub_height + wg_height/2), axes):
    sim.viz_eps_2D(normal=val, position=pos, ax=ax, cbar=True)

In [ ]:
ax1, ax2 = sim.viz_source(mode_source)
ax1.set_xlim((0, 0.1e-12))  # note the pulse extends far beyond this time, adjust lims to inspect
plt.show()

In [ ]:
sim.compute_modes(mode_source, Nmodes=2)
sim.viz_modes(mode_source)

# Mode 0 is the Ey dominant mode, so we choose that
mode_ind = 0
sim.set_mode(mode_source, mode_ind)

## Run Simulation

Run the simulation and plot the field patterns

In [ ]:
# create a project, upload to our server to run
project = web.new_project(sim.export(), task_name='grating_coupler')
task_id = project['taskId']
web.monitor_project(task_id)

In [ ]:
# download the results and load into the original simulation
print('downloading results...')
web.download_results(task_id, target_folder='out')
print('done\n')
sim.load_results('out/monitor_data.hdf5')
with open("out/tidy3d.log") as f:
     print(f.read())

In [ ]:
fig, axes = plt.subplots(3, 1, tight_layout=True, figsize=(10, 8))
for monitor, cpmp, ax in zip([plane_monitor, rad_monitor, near_field_monitor], 'yyy', axes):
    sim.viz_field_2D(monitor, comp=cpmp, ax=ax, cbar=True)

## Far Field Projection
Now we use the Near2Far feature of Tidy3D to compute the anglular dependence of the far field scattering based on the near field monitor.

In [ ]:
# create range of angles to probe (note: polar coordinates, theta = 0 corresponds to vertical (z axis))
num_angles = 1101
thetas = np.linspace(-np.pi/2, np.pi/2, num_angles)

# make a near field to far field projector with the near field monitor data
n2f = td.Near2Far(sim.data(near_field_monitor))

# loop through angles and record the scattered cross section
Ps = np.zeros(num_angles)
for i in range(num_angles):
    Ps[i] = n2f.get_radar_cross_section(thetas[i], 0.0)

In [ ]:
# plot the angle dependence
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(5,5))
ax.plot(thetas, Ps, label='measured')
ax.plot([design_theta_rad, design_theta_rad], [0, np.max(Ps)*0.7], 'r--', alpha=0.8, label='design angle')
ax.set_theta_zero_location("N")
ax.set_yticklabels([])
ax.set_title("Scattered Cross-section (arb. units)", va='bottom')
plt.legend()
plt.show()

In [ ]:
theta_expected = np.arcsin(np.abs(neff - wavelength / pitch))
print(f'expect angle of {(theta_expected * 180 / np.pi):.2f} degrees')
i_max = np.argmax(Ps)
print(f'got maximum angle of {(thetas[i_max] * 180 / np.pi):.2f} degrees')

The agreement between the target angle and the actual emission angle of the coupler is very good. The small difference comes from the fact that the design is very sensitive to the value of the effective index that we use in the coupler region, and that value depends on which waveguide height we pick in that region: the one with the grating comb, or without. In our setup, we used a thickness that is at the mid-point, but this is a heuristic choice which results in the small final mismatch in angles observed here.